In [2]:
from notebook.services.config import ConfigManager
cm = ConfigManager()

In [3]:
from notebook.services.config import ConfigManager
cm = ConfigManager()

In [4]:
cm.update('livereveal', {
#              'width': 1000,
#              'height': 600,
#              'scroll': True,
})

{'width': 1000, 'height': 600, 'scroll': True}

In [ ]:
# Select a read only variable which gives a new output every now and then 

<h1> Introduction to Bluesky and Ophyd </h1>

Author: Pierre Schnizer


<h2> Literature </h2>

<h3> Python knowledge </h3>

* classes and inheritance
* generators  
* decorators
* asyncio
* Global Intepreter Lock



<h2> Literature </h2>

<h3> Python </h3>
The author assumes that the reader is familiar with python language version 3. 
Bluesky and Ophyd make have use of:

* asyncio
* generators  
* decorators
* Global Intepreter Lock



<h3> Abstraction </h3>

Most of these implementation details are abstracted in an elegant manner. 
But ocassioally, in particular when debugging 
a faulty device driver, it can be helpful to be awaare of the general concepts of asynchronous programming.

Generators are used by bluesky plans or plan stubs. If plans are used, which are provided by bluesky,
the user does not need to worry about these details.

If one wants to implement ophyd device drivers, the concept of classes should be understood.

<h3> Bluesky and ophyd links </h3>

* Bluesky and ophyd are available from https://github.com/bluesky/bluesky
* Documentation: https://blueskyproject.io/bluesky/
* Tutorial: https://github.com/bluesky/tutorial/
* Chat https://gitter.im/NSLS-II/DAMA

<h2> General Concepts </h2>

Bluesky is used with already two meanings:
* Bluesky collaboration: a set of different modules used to simplify taking data within the EPICS control system environement
* Bluesky as the python module, which provides the RunEngine and plans

In the following I will use "Bluesky" as reference to the module. If I want to refer to the whole package, I refer to it
as "Bluesky measurement orchestration"



<h3> Bluesky build up</h3>

Bluesky collaboration uses the following concpets:
* A control system (EPICS) is responsible for taking data
* This control system provides asynchronos and parallel access to the devices using variables
* It separates device abstraction from measurement plans
* In this concept:

    * Ophyd allows making devices accessible abstracted as classes with different methods
    * Bluesky provides measurement plans and the run engine
    * The run engine executes the plans by calling appropriate methods of the involved devices. 
      (This description is a bit imprecise and will be detailed further below)




<h3> Asynchronous access</h3>

At this stage, one could still directly access the different process variables. But the run engine
assures that these are accessed in an asynchronous fashion. How this works is detailed below.

Furthermore:
* these modules (as much as python) are not real time software (and in particular not 
  *hard* real time software)
* but it has the tools to:
  * set up different hard real time devices
  * to get them going 
  * and to collect their data in the end
* It is right to control a small or medium size experiments

Your millage may vary ;-)

<h3> Device driver example </h3>

Let's look to a simple device driver. For the beginning let's assume we have a read only
Epics variable with name 'sig:ro'.


In [10]:
from ophyd import Device
from ophyd.status import Status

class ReadonlyDevice(Device):
    """A simple sample device
    
    This devivce is solely used to describe the methods
    """
    ro = Cpt(EpicsSignalRO, name = 'sig:ro')

If nothing fancy is required that would be sufficient. If we now want to read the data 
lets say 5 times the following code would do the job

In [ ]:
from bluesky import plans as bps, RunEngine 

r = ReadonlyDevice(name = sig)

RE = Runengine()
RE(bps.count(r, 5))

This is rather opaque, so lets see what is happening under the hood:

This we can do if we use the same code as above and enable logging

In [16]:
from bluesky import plans as bps, RunEngine 

# Check that debug print works!
# RE = RunEngine({})
# RE.log.setLevel('DEBUG')


r = ReadonlyDevice(name = 'sig')

RE(bps.count(r, 5))

TypeError: __init__() missing 1 required positional argument: 'read_pv'

Now let'S look to the output. There is quite some output being presented. 
What is happending? Let's look under the hood:

XXX check if that is right
* There is a call to trigger made
* There is a call to read

So what's the difference?
* The method trigger is allowed to take time (but not computation time!) 
* The method read is used to generate data but not from the device

What are the two methods doing:
* The method trigger actually "triggers" that a device is read.
  It is expected that during this call the data from some remote device are transfered to the
  local device
* The method read takes care to handle the data over to the run engine

So let's intercecpt the methods to get a better idea of that

In [18]:
from ophyd import Device, Component as Cpt, EpicsSignalRO

class ReadonlyDevice(Device):
    """A simple sample device
    
    This devivce is solely used to describe the methods
    """
    ro = Cpt(EpicsSignalRO, name = 'sig:ro')
    
    def trigger(self):
        status = super().trigger()
        self.log.debug(f'Super method trigger returned {status}')
        return status
    
    def read(self):
        result = super().read()
        self.log.debug(f'Read returned {result}')
        return r

Now we see a difference:
* trigger returns a status object
* while read returns data

What's a status object:
* a delayed return value
* think of the tokens you get in self service restaurants: they buzz as soon as your order is finished


This is the general concept of ophyd and bluesky:
* methods returning data are expected to execuate without any delay
* methods returning status objects are allowed responding with some delay.
  (but not to take computation. How to do that is detailed furhter down)
  
What'S the status object: well think of the new resteraunts that give you a token when you order: 
the token rings as soon as you are finished.

I'll detail that in an other example:

In [21]:
from ophyd import Device
from ophyd.status import DeviceStatus

class ReadonlyDevice(Device):
    """A simple sample device
    
    This devivce is solely used to describe the methods
    
    Warning:
        This code is not properly made
    """
    ro = Cpt(EpicsSignalRO, name = 'sig:ro')
    
    def trigger(self):
        """
        
        Warning:
         This code is not the most elegant one ....
          See further down for a simpler one
        """
        cid = None
        status = Devicestatus()
        def ensure_new_data(*args, value = None, old_value = None, **kwargs):
                self.ro.clear_sub(ensure_new_data)
                status._finished()
                
                
        self.ro.subscibe(ensure_new_data)
        return status

The trigger method shows:
* a status object is created. 
* a callback is subscribed to the variable
* a status object is returned.

How is that code executed:
* The run engine will call trigger on the method  of this object
* it will recieve a status object. 

The run eninge will take the status object and wait for it to complete. If different devices are to be checked it 
will wait that the last one has completed. In this manner it ensures that the different variables are handled in 
parallel. 

Please note: typically this code uses epics.PV. Thus this approach is not suitable for a large set of variables!


This code can be further simplified if we use SubscriptionStatus instead of device status. Let's implement 
the smae code again


In [23]:
from ophyd import Device
from ophyd.status import SubscriptionStatus

class ReadonlyDevice(Device):
    """A simple sample device
    
    """
    ro = Cpt(EpicsSignalRO, name = 'sig:ro')
    
    def trigger(self):
        """Ensure that new data has arrived
        """
        def cb(*args,  **kwargs):
            return True
        
        status = SubcriptionStatus(self.ro, cb, run = False)
        return status

So now we arrive at some pretty straight forward code

When trigger is called, the object SubscriptionStatus will subscribe the function (here ensure_run_once) to self.ro.
Then the callback 'cb' is called every time when new data arrrive at signal self.ro.

The keyword argument run=False, ensures that the method is only called the first time when  new data is arrived.
It defaults to True. In this case it will be run immediately after the callback 'cb' has been subscribed 
to the device.

Please note again, that all these triggers are expected to behave well. IF you need to wait or to do processing,
see further down how to do that



Now we have used a very simple device Now let's see if we have a device with two signals

In [24]:
from ophyd import Device
from ophyd.status import SubscriptionStatus

class ReadonlyDevice(Device):
    """A simple sample device
    
    """
    ro = Cpt(EpicsSignalRO, name = 'sig:ro')
    ro2 = Cpt(EpicsSignalRO, name = 'sig2:ro')

Now lets run the same simple example again as above:

In [26]:
from bluesky import plans as bps, RunEngine 

# Check that debug print works!
#RE = RunEngine()
#RE.log.setLevel('DEBUG')


r = ReadonlyDevice(name = 'sig')

RE(bps.count(r, 1))

NameError: name 'sig' is not defined

The  important thing to note now:
* We can see one trigger
* and two read calls.

That's the policy of bluesky (as far as I understand)
* It will call trigger only for the top device. It is 
  the responsiblity of this device to delegate the triggers to other objects
  if required.
  
* Reads are then transfered to all other devices.

Please note that devices can be generated as a tree. The user can call handle 
either the top device or a leaf of the tree to the run engine. The run engine
will call the method trigger only on the object passed to it.

In this manner the device driver writer can organise how the device is read.
For example, let's ensure that the data of the second signal has arrived only
after the signal for the first one has arrived.

This can be implemented in different manners. The one I chose is used to demonstrate
the add_callback method of the status object.

In [33]:
from ophyd import Device, Component as Cpt, EpicsSignalRO
from ophyd.status import SubscriptionStatus, AndStatus

class ReadonlyDevice(Device):
    """Ensure data of signal ro2 are always newer than for signal ro. 
    """
    ro = Cpt(EpicsSignalRO, name = 'sig:ro')
    ro2 = Cpt(EpicsSignalRO, name = 'sig2:ro')
    
    def trigger(self):
        
        ro_done = False
        def cb(*args, **kwargs):
            return True
    
        def cb2(*args, **kwargs):
            #nonlocal ro_done
            if ro_done:
                return True
            return False
    
        def delayed_call(*args, **kwargs):
            #nonlocal ro_done
            ro_done = True
        
        stat2 = SubscriptionStatus(self.ro, cb2, run = False)
        stat = SubscriptionStatus(self.ro, cb, run = False)
        stat.add_callback(delayed_call)
        s = AndStatus(stat2, stat)
        return s

This code could be chained further on. Such code can extended further on and stacked. 
Based on my experience: if the device needs more handling I recommend to consider to 
base the code on a state engine.

In [34]:
from ophyd import Device, Component as Cpt, EpicsSignal, Signal, DeviceStatus

class AccumulatingSignal(Device):
     target = Cpt(EpicsSignal, 'thermo:I')
     window_size = Cpt(Signal, value=5)
     last_read = Cpt(Signal, value=[])     


     def trigger(self):

          dbuffer = []
          count = 0
          target_N = self.window_size.get()
          status = DeviceStatus(self)

          def accumulating_callback(value, **kwargs):
               if status.done:
                    self.target.clear_sub(accumulating_callback)
               nonlocal count

               dbuffer.append(value)
               count += 1

               if count >= target_N:
                    self.last_read.put(dbuffer[:target_N])
                    self.target.clear_sub(accumulating_callback)                         
                    status._finished()

          self.target.subscribe(accumulating_callback, run=False)          

          return status

     def put(self, *args, **kwargs):
           raise Exception("no, read only")

<h3> Summary </h3>

So to sum up:

Bluesky run eninge is based on cooperative multi tasking. It assumes that all calls to ophyd device require only a short time to run.
Methods that return a status object, are expected to return the status object immediately. 

* bluesky run engine $\to$ cooperative multi task
* ophyd device driver:
   * typically: methods are expected short sequence of CPU
   * methods returning status objects:
      * return staatus object immediatly
      * status object method '_finished()' called when done
      * or use 'SubscriptionStatus'
      * up to now: react to callbacks e.g. triggered by data changed 
        by epics

<h2>Ophyd signals: everything is a signal </h2>  

Compariosn: Unix everything is a file

Ophyd:
* Everything is a signal
* Signals are used to create events
* Lifeplots can use them
* Bluesky run engine uses signals to gather device information
* See further down on event model to understand how it simplifies
  measurements

<h2> Settable devices </h2>

The description above concentrated on generating devices, which allow only reading data. 
Let's see how to implement that. Here I will not go into the basis but show how it can be done
using the already implemented bases of ophyd.

(I did all the details above as I wanted to illustrate the difference of direct calls to delayed 
calls).

For ophyd settable devices are consideed as "motors". Any device that has a set value and a read value falls into 
this category. Personally I prefer to think of an actuator instead of a motor.

Ophyd has several implementations, which facilitate implementing actuators:
* Motors
* PVPositioner
* PVPositionerPC

Let's implement a PVPositioner (see also https://github.com/TMsangohan/bact2/blob/66d4bea4f13507b1b82ccb3577fc6a7bfc4b627c/examples/exp/mono_chromator.py#L1)

In [36]:
from ophyd import EpicsSignal, EpicsSignalRO, PVPositioner, Component as Cpt

class Monochromator( PVPositioner ):
    setpoint    = Cpt(EpicsSignal,   ':monoSetEnergy')
    readback    = Cpt(EpicsSignalRO, ':monoGetEnergy')
    done        = Cpt(EpicsSignalRO, ':multiaxis:running')

<h3> PV positioner </h3>

Ophyd PV Positioner: requires three variables:
* setpoint: a variable that describes the desired value
* readback: a variable that descrives the read back value
* done: a variable that indicates if the move is finished

**Note**: ophyd will consider the move finished when the variable done is set to **True**
Setpoint is only used to set the value, readback to read the value
Based on these signals the user can get feedback on how long it will take that the 
move is finished

<h3> PV positioner </h3>

Ophyd PV Positioner: more functionality:
* done: indication value can be set to **False**
* stop value: used to request a stop to the actuator

See an implementation for a BESSY II monochromator for a idea of it

In [37]:
from ophyd import EpicsSignal, EpicsSignalRO, PVPositioner, Component as Cpt

class Monochromator( PVPositioner ):
    setpoint    = Cpt(EpicsSignal,   ':monoSetEnergy')
    readback    = Cpt(EpicsSignalRO, ':monoGetEnergy')
    done        = Cpt(EpicsSignalRO, ':multiaxis:running')
    done_val = 0
    stop_signal = Cpt(EpicsSignal,   ':multiaxis:stop')
    stop_val = 1

<h3>Adapting it further to our needs</h3
    
The monochromator now is fairly nicely  fulfilling our needs. But we can go further....

Ophyd follows the idea of intelligent devices:
* Settle time: how long does it take a device to stabilise after 
  the movement has signalled finished (e.g. a bar stops osszilating)
* Time out: maximum time that the movement is allowwed to take

**NB** these are variables of the status object. Thus' these can be 
computed if required....


In [38]:
class _Monochromator( PVPositioner ):
    setpoint    = Cpt(EpicsSignal,   ':monoSetEnergy')
    readback    = Cpt(EpicsSignalRO, ':monoGetEnergy')
    done        = Cpt(EpicsSignalRO, ':multiaxis:running')
    done_val = 0
    stop_signal = Cpt(EpicsSignal,   ':multiaxis:stop')
    stop_val = 1


    def trigger(self):
        """Trigger only after new data has arrived
        Deactivated not to interfere with :class:`PVPositioner`
        
        Warning:
            Untested code!!!
            
        Better test it first
        """
        status = super().trigger()
        status.set_settle

        stat_rdbk = SubscriptionStatus(self.readback, cb_rdbk, run = False, timeout = 1, settle_time = 2)
        return stat_rdbk

class Monochromator( Device ):
    dev = Cpt(_Monochromator, 'u171dcm1', name = 'mc', egu = 'eV')

<h3> Everything is a signal </h3>

Settle time and timeouts are values, which are not necessarily documented. But 
this can be handled too


In [41]:
class _Monochromator( PVPositioner ):
    setpoint    = Cpt(EpicsSignal,   ':monoSetEnergy')
    readback    = Cpt(EpicsSignalRO, ':monoGetEnergy')
    done        = Cpt(EpicsSignalRO, ':multiaxis:running')
    done_val = 0
    stop_signal = Cpt(EpicsSignal,   ':multiaxis:stop')
    stop_val = 1

    timeout    = Cpt(Signal, 'timeout', default=1.0)
    settle_time = Cpt(Signal, 'settle_time', default=2.0), 
    
    def trigger(self):
        """Trigger only after new data has arrived
        Deactivated not to interfere with :class:`PVPositioner`
        
        Warning:
            Untested code!!!
            
        Better test it first
        """
        status = super().trigger()
    
        stat_rdbk = SubscriptionStatus(self.readback, cb_rdbk, run = False, 
                                       timeout = self.timeout.value, 
                                       settle_time = self.settle_time.value)
        stat_rdbk.add_callback(status)
        return stat_rdbk

<h3> Marking signals as cofiguration signals</h3>

Blueskys RunEngin will emit event documents. The setup is described in XXXX.

As a rule of thumb:
* Ophydd devices can contain a tree of devices
* Bluesky inspects the devices and read all signals
* The information of all signals will be added to each event



In [42]:
class _Monochromator( PVPositioner ):
    # Class members skipped
    _default_configuration_attrs = ('timeout', 'settle_time')

In this case the timeout and settle time variables are read out only at the beginning of the run.

<h3> A slow moving device </h3>

If we have a slow moving device, perhaps we want to compute the acceptable 
time for the movement depending on where we are 

In [43]:
class SlowMovingDevice ( PVPositioner ):
    setpoint = Cpt(EpicsSignal,   ':set')
    readback = Cpt(EpicsSignalRO, ':rbk')
    done     = Cpt(EpicsSignalRO, ':done')
    
    timeout    = Cpt(Signal, 'timeout', default=1.0)
    settle_time = Cpt(Signal, 'settle_time', default=2.0), 

    
    def computeMovementTime(self, current, target):
        """
        Todo:
            compute settle time
        """
        
        return settle_time
    
    def set(self, value):
        """
        
        WARNING:
            Untested code!
        """
        current = self.readback.value
        movement_time = self.computeMovementTime(current, value)
        # how much to add to the pure movement time?
        # typically a percentage and an additional value
        setlle_time = movement_time
        self.settle_time.set(settle_time)
        
        # create a status object or add the settle time to the 
        # status object